In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import smartscan as ss

In [ ]:
sm = ss.SmartScan('localhost', 12345)

In [ ]:
sm.sgm4.get_scan_info()

In [ ]:
vm.init_scan_from_file(
    filename="D:\data\SGM4 - 2022 - CrSBr\data\Kiss05_15_1.h5"
)


In [ ]:
vm.create_file(mode='x',)

In [ ]:
scanner = ss.SGM4FileReader(vm.target_file_name)
scanner.open()

In [ ]:
# # %%timeit
# for i in range(5):
#     vm.current_pos = random_point = np.array([
#         random.uniform(vm.limits[0][0], vm.limits[0][1]),
#         random.uniform(vm.limits[1][0], vm.limits[1][1])
#     ])
#     vm.measure()

In [ ]:
scanner.positions

In [ ]:
i=1
f'Kiss05_15_1_virtual_{i:03.0f}.h5'

In [ ]:
scanner.get_data(slice(None,None,None)).shape

# GPcam

In [ ]:
from gpcam.gp_optimizer import GPOptimizer
from numpy.random import default_rng

import plotly.graph_objects as go
import numpy as np

# initialize scan with center point

In [ ]:
scanner.limits

In [ ]:
vm.current_pos = vm.nearest_position_on_grid(
    [np.mean(l) for l in scanner.limits]
)
print(vm.current_pos)


In [ ]:
vm.measure()

In [ ]:
def find_next(n,points,values,gp):
    gp.tell(points,values)
    gp.train_gp(
        hyperparameter_bounds=np.array([[0.001,1e9],[1,1000],[1,1000]]),
        pop_size = 20,
        tolerance = 1e-6,
        max_iter = 2
    )
    new = gp.ask(
        position = None, 
        n = n, 
        acquisition_function = "shannon_ig", 
        bounds = None,
        method = "global", 
        pop_size = 20, 
        max_iter = 20, 
        tol = 10e-6, 
        x0 = None, 
        dask_client = False
    )
    return new


# initialize

In [ ]:

points = scanner.positions
values= np.array([v.sum() for v in scanner.get_data(slice(None,None,None))])
assert len(points) == len(values),"points and values don't match"
print(points.shape,values.shape)


In [ ]:
gp = GPOptimizer(
    input_space_dimension=2, 
    input_space_bounds = scanner.limits
)
gp.tell(points,values)
gp.init_gp(np.array([4.71907062e+06, 4.07439017e+02, 3.59068120e+02]))
gp.train_gp(
    hyperparameter_bounds=np.array([[0.001,1e9],[1,1000],[1,1000]]),
    pop_size = 20,
    tolerance = 1e-6,
    max_iter = 2
)


In [ ]:
points = scanner.positions
values = np.array([v.sum() for v in scanner.get_data(slice(None,None,None))])
next = find_next(4,points,values,gp)
print(next)

In [ ]:
scanner.limits